In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
os.chdir('data/entire_celeb_dataset')
celebs = os.listdir(os.getcwd())
celebs

['Aamir_Khan',
 'Abhay_Deol',
 'Abhishek_Bachchan',
 'Aftab_Shivdasani',
 'Aishwarya_Rai',
 'Ajay_Devgn',
 'Akshaye_Khanna',
 'Akshay_Kumar',
 'Alia_Bhatt',
 'Ameesha_Patel',
 'Amitabh_Bachchan',
 'Amrita_Rao',
 'Amy_Jackson',
 'Anil_Kapoor',
 'Anushka_Sharma',
 'Anushka_Shetty',
 'Arjun_Kapoor',
 'Arjun_Rampal',
 'Arshad_Warsi',
 'Asin',
 'Ayushmann_Khurrana',
 'Bhumi_Pednekar',
 'Bipasha_Basu',
 'Bobby_Deol',
 'Deepika_Padukone',
 'Disha_Patani',
 'Emraan_Hashmi',
 'Esha_Gupta',
 'Farhan_Akhtar',
 'Govinda',
 'Hrithik_Roshan',
 'Huma_Qureshi',
 'Ileana_DCruz',
 'Irrfan_Khan',
 'Jacqueline_Fernandez',
 'John_Abraham',
 'Juhi_Chawla',
 'Kajal_Aggarwal',
 'Kajol',
 'Kangana_Ranaut',
 'Kareena_Kapoor',
 'Karisma_Kapoor',
 'Kartik_Aaryan',
 'Katrina_Kaif',
 'Kiara_Advani',
 'Kriti_Kharbanda',
 'Kriti_Sanon',
 'Kunal_Khemu',
 'Lara_Dutta',
 'Madhuri_Dixit',
 'Manoj_Bajpayee',
 'Mrunal_Thakur',
 'Nana_Patekar',
 'Nargis_Fakhri',
 'Naseeruddin_Shah',
 'Nushrat_Bharucha',
 'Paresh_Rawal',
 'P

In [3]:
if os.path.isdir('train/Aamir_Khan/') is False: 
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')

    for c in celebs:
        shutil.move(f'{c}', 'train')
        os.mkdir(f'valid/{c}')
        os.mkdir(f'test/{c}')

        valid_samples = random.sample(os.listdir(f'train/{c}'), 30)
        for i in valid_samples:
            shutil.move(f'train/{c}/{i}', f'valid/{c}')

        test_samples = random.sample(os.listdir(f'train/{c}'), 5)
        for j in test_samples:
            shutil.move(f'train/{c}/{j}', f'test/{c}')
os.chdir('../..')

In [4]:
train_path = 'data/entire_celeb_dataset/train'
valid_path = 'data/entire_celeb_dataset/valid'
test_path = 'data/entire_celeb_dataset/test'

In [5]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
                .flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
                .flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
               .flow_from_directory(directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 5164 images belonging to 100 classes.
Found 3000 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


In [39]:
mobile = tf.keras.applications.mobilenet.MobileNet(weights = 'imagenet', include_top = False)
for layer in mobile.layers[:-2]:
    layer.trainable = False
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 

In [40]:
top_model = mobile.output

In [41]:
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(1024, activation='relu')(top_model)
top_model = Dense(1024, activation='relu')(top_model)
top_model = Dense(512, activation='relu')(top_model)
top_model = Dense(units=100, activation='softmax')(top_model)

In [42]:
model = Model(inputs=mobile.input, outputs=top_model)

In [43]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=8,
            verbose=2
)

Epoch 1/8
517/517 - 124s - loss: 4.2620 - accuracy: 0.0562 - val_loss: 4.0521 - val_accuracy: 0.0530
Epoch 2/8
517/517 - 120s - loss: 3.6243 - accuracy: 0.1346 - val_loss: 3.5793 - val_accuracy: 0.1493
Epoch 3/8
517/517 - 121s - loss: 3.0887 - accuracy: 0.2459 - val_loss: 3.3745 - val_accuracy: 0.1727
Epoch 4/8
517/517 - 126s - loss: 2.5588 - accuracy: 0.3679 - val_loss: 2.9850 - val_accuracy: 0.2863
Epoch 5/8
517/517 - 122s - loss: 2.1215 - accuracy: 0.4669 - val_loss: 2.9773 - val_accuracy: 0.2967
Epoch 6/8
517/517 - 121s - loss: 1.7479 - accuracy: 0.5515 - val_loss: 2.7563 - val_accuracy: 0.3363
Epoch 7/8
517/517 - 124s - loss: 1.4599 - accuracy: 0.6139 - val_loss: 2.6677 - val_accuracy: 0.3577
Epoch 8/8
517/517 - 124s - loss: 1.1868 - accuracy: 0.6938 - val_loss: 2.7019 - val_accuracy: 0.3660


In [45]:
test_labels = test_batches.classes

In [46]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [47]:
predictions

array([[2.1271306e-01, 1.5263165e-02, 1.2360231e-04, ..., 1.7781351e-03,
        9.7603254e-07, 7.1601150e-08],
       [2.4016027e-01, 8.8515570e-03, 7.3640118e-03, ..., 6.2439952e-02,
        2.0241573e-02, 1.0653135e-03],
       [1.1246684e-01, 3.7985502e-03, 2.0980688e-03, ..., 2.5034774e-02,
        2.3919738e-04, 3.2948737e-05],
       ...,
       [3.0291551e-06, 5.1446710e-07, 1.6248425e-05, ..., 3.6999854e-04,
        1.2286303e-03, 5.7612377e-01],
       [6.9570946e-05, 2.7559069e-04, 2.1874640e-04, ..., 2.0464913e-04,
        9.0272783e-04, 3.1261921e-03],
       [3.2070879e-05, 6.6998065e-05, 4.2046144e-04, ..., 6.0344432e-03,
        3.4423738e-03, 1.0808028e-01]], dtype=float32)

In [48]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [49]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.figure(figsize=(20,20))

In [50]:
test_batches.class_indices

{'Aamir_Khan': 0,
 'Abhay_Deol': 1,
 'Abhishek_Bachchan': 2,
 'Aftab_Shivdasani': 3,
 'Aishwarya_Rai': 4,
 'Ajay_Devgn': 5,
 'Akshay_Kumar': 6,
 'Akshaye_Khanna': 7,
 'Alia_Bhatt': 8,
 'Ameesha_Patel': 9,
 'Amitabh_Bachchan': 10,
 'Amrita_Rao': 11,
 'Amy_Jackson': 12,
 'Anil_Kapoor': 13,
 'Anushka_Sharma': 14,
 'Anushka_Shetty': 15,
 'Arjun_Kapoor': 16,
 'Arjun_Rampal': 17,
 'Arshad_Warsi': 18,
 'Asin': 19,
 'Ayushmann_Khurrana': 20,
 'Bhumi_Pednekar': 21,
 'Bipasha_Basu': 22,
 'Bobby_Deol': 23,
 'Deepika_Padukone': 24,
 'Disha_Patani': 25,
 'Emraan_Hashmi': 26,
 'Esha_Gupta': 27,
 'Farhan_Akhtar': 28,
 'Govinda': 29,
 'Hrithik_Roshan': 30,
 'Huma_Qureshi': 31,
 'Ileana_DCruz': 32,
 'Irrfan_Khan': 33,
 'Jacqueline_Fernandez': 34,
 'John_Abraham': 35,
 'Juhi_Chawla': 36,
 'Kajal_Aggarwal': 37,
 'Kajol': 38,
 'Kangana_Ranaut': 39,
 'Kareena_Kapoor': 40,
 'Karisma_Kapoor': 41,
 'Kartik_Aaryan': 42,
 'Katrina_Kaif': 43,
 'Kiara_Advani': 44,
 'Kriti_Kharbanda': 45,
 'Kriti_Sanon': 46,
 'Kun

In [51]:
cm_plot_labels = celebs[:]
#plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
fig = plt.figure(figsize=(36,36), dpi = 480)
sns.heatmap(cm, annot = True)

<AxesSubplot:>